# **Business Understanding:**
***
电信用户流失会对服务商造成很大影响。现收集了用户的数据，包括用户性别、用户属性、已开通服务等信息，已标记出流失用户。分析已流失用户特征，并预测未来用户是否会流失。

# **Data Understanding:**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as mtick

In [ ]:
tele=pd.read_csv('../input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv')
tele.info()
tele.head()

## 1. 数据集信息：
1. 上个月内离开的客户–该列称为“客户流失”
2. 每个客户已签署的服务-电话，多条线路，互联网，在线安全，在线备份，设备保护，技术支持以及流电视和电影
3. 客户帐户信息–他们成为客户的时间，合同，付款方式，无纸化账单，每月费用和总费用
4. 有关客户的人口统计信息-性别，年龄范围以及他们是否有伴侣和受扶养人
***



## 2.处理缺失数据：
查看数据发现，原数据集中SeniorCitizen,TotalCharges数据格式不正确。

In [ ]:
#修改TotalCharges为数据形式
tele.TotalCharges = pd.to_numeric(tele.TotalCharges,errors='coerce') #errors='coerce'若无法转化为要求格式，则用NaN填充
#转换数据格式后，totalcharges列有11个记录有缺失值，把他们去除
tele.dropna(inplace = True)
#id列与标的变量无关，去掉id列
df1=tele.iloc[:,1:]
#把churn转化为0，1形式
df1['Churn'].replace(to_replace='Yes',value=1,inplace=True) #inplace=True为直接改变原数据
df1['Churn'].replace(to_replace='No',value=0,inplace=True)

In [ ]:
#转化为虚拟变量，为讨论相关性
tele_dummies = pd.get_dummies(df1)

## 3.相关性
要讨论变量与标的变量之间的相关性，热力图不现实，这里变量太多了 

plt.subplots(figsize=(30, 30))
sns.heatmap(tele_dummies.corr(),annot=False)
plt.show()

In [ ]:
#因此选用柱形图
plt.figure(figsize=(15,8))
tele_dummies.corr()['Churn'].sort_values(ascending = False).plot(kind='bar') #这个语句需要学习

## 4.小结
在进一步分析各变量之前，可以通过上图对各变量与标的变量的关系有大致了解。
月包合同，无网络安全服务，无技术支持与流失用户有正向关系，而使用时长，两年期合同，无网络服务与流失用户有负向联系。
其次，性别、手机服务、多线路这三个变量与标的变量相关性几乎为零。

# **Data exploration**

## Categorical Variables
进一步探讨前，把binary variables的值转化为1，0

In [ ]:
cols =[cols for cols in df1.columns if df1[cols].dtype=='object' or cols =='SeniorCitizen']  #只挑选object的变量
df1_cols = df1[cols].copy() #需要转换的变量组成一个dataframe

In [ ]:
for i in cols:
    if df1_cols[i].nunique() == 2:
        df1_cols[i],_=pd.factorize(df1_cols[i])
    else:
        df1_cols = pd.get_dummies(df1_cols,columns=[i])

print(df1_cols.head(3))

### 各分类型变量与标的变量的关系

In [ ]:
cols =[cols for cols in df1.columns if df1[cols].dtype=='object' or cols =='SeniorCitizen'] # 挑选分类型变量
# 画图
fig,axes = plt.subplots(nrows=4,ncols=4,figsize=(50,50))
axis_name='Percentage of Customers'
for i, c in enumerate(cols):
    g = df1.groupby(c)['Churn'].value_counts()/len(df1) #求每个变量流失与留存用户的各个值的个数
    g = g.to_frame().rename({'Churn':axis_name},axis=1).reset_index()
    if i<4:
        ax=sns.barplot(x=c,y=axis_name,hue='Churn',data=g,ax=axes[i,0])
    elif i>=4 and i< 8:
        ax=sns.barplot(x=c,y=axis_name,hue='Churn',data=g,ax=axes[i-4,1])
    elif i>=8 and i <12:
        ax=sns.barplot(x=c,y=axis_name,hue='Churn',data=g,ax=axes[i-8,2])
    elif i<16:
        ax=sns.barplot(x=c,y=axis_name,hue='Churn',data=g,ax=axes[i-12,3])
    ax.set_title(c)

***
### 小结

## Tenure, MonthlyCharges, TotalCharges
这三个数值型变量一起讨论

In [ ]:
sns.distplot(df1['tenure'],hist=True,kde=False) #kde是否有拟合线，这里可以没有

In [ ]:
#先定义一个kdeplot
def kdeplot(feature):
    plt.figure(figsize=(9, 4))
    plt.title("KDE for {}".format(feature))
    ax0 = sns.kdeplot(df1[df1['Churn'] == 0][feature], color= 'navy', shade=True,label= 'Churn: No')
    ax1 = sns.kdeplot(df1[df1['Churn'] == 1][feature], color= 'orange', shade=True,label= 'Churn: Yes')
    plt.legend(['Not','Churn'])

kdeplot('tenure')
kdeplot('MonthlyCharges')
kdeplot('TotalCharges')

这里可看出用户加入时长多集中在1个月或者是72个月，新加入的用户更容易流失，老用户不容易流失
其中，入网时长小于20个月的客户流失率最高

月付费高的客户更容易流失，月付费低的不容易流失。月付费高于60客户会大量流失。

这里发现总付费与入网时长有关，探究他们的关系

In [ ]:
df1['ChargesPerMonth']=df1['TotalCharges']/df1['tenure']
kdeplot('ChargesPerMonth')

In [ ]:
df1['Month_diff']=df1['MonthlyCharges']-df1['ChargesPerMonth']
kdeplot('Month_diff')

In [ ]:
df1 = df1.iloc[:,:-2] #删除新增的两列

### 小结
发现总花费/时长与每月花费的分布一致。每月话费是一个重要变量

In [ ]:
#这里只看流失客户
cols = ["OnlineSecurity", "OnlineBackup", "DeviceProtection", "TechSupport", "StreamingTV", "StreamingMovies"]
df2 = df1[(df1.InternetService!='No') & (df1.Churn ==1)]
df2 = pd.melt(df2[cols]).rename({'value': 'Has service'}, axis=1)
print(df2)
plt.figure(figsize=(10,5))
ax = sns.countplot(data=df2, x='variable', hue='Has service', hue_order=['No', 'Yes'])

订阅前四种服务的客户不容易流失，流媒体服务对churn的影响不大

### 5. Contract and Payment

In [ ]:
barplot_percentages('Contract')

In [ ]:
barplot_percentages('PaperlessBilling')

In [ ]:
plt.figure(figsize=(10,5))
barplot_percentages('PaymentMethod')

月期合同的用户更容易流失，电子账单的用户更容易流失

In [ ]:
#查看不同支付方式与monthlycharges的关系
ax = sns.catplot(y="Churn", x="MonthlyCharges", row="PaymentMethod", kind="box", data=df1, height=1.5, aspect=4, orient='h')

邮寄付费的用户每月话费最少，在同一种支付方式中，话费高的客户更容易流失 

### 6.结论

# **Building Models and evaluations**

分别使用逻辑回归，随机森林，svm模型

## 1. LogisticRegression 

In [ ]:
#转化为dummy variable
df1_dummies=pd.get_dummies(df1)

In [ ]:
X=df1_dummies.drop(columns='Churn').values
y=df1_dummies['Churn'].values

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=42)

log=LogisticRegression()
log.fit(X_train,y_train)
y_pred=log.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(log.score(X_test, y_test))
print(log.coef_)

逻辑回归模型的准确率为79.6%，流失用户检测的精准度为64%，即检测出为流失用户的群体中有64%为真的流失用户。

召回率为53%，即所有流失用户中被检测出的比率为53%。

对于流失用户预测来说，type2error（实际为流失用户，但预测为留存用户）造成的商业损失比type1error大，此模型的召回率很低，因此需要提高召回率。

此数据集是不均衡数据集，可以采用增加权重的方式来增加流失用户的数据量

In [ ]:
log_weight=LogisticRegression(class_weight={0:1,1:2})
log_weight.fit(X_train,y_train)
y_pred=log_weight.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(log.score(X_test, y_test))
print(log.coef_)

召回率提高了20%，精准度下降了8%。注意，不能一味只关注召回率，为达到召回率尽可能高的目的（100%），可以把所有样本都变成1。
不适合使用准确度、召回率来评估不均衡数据集的表现，这里auc,roc更适合。
还需讨论的问题：特征选择，随机森林的参数c选择，交叉验证选模型

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
y_pred_prob = log.predict_proba(X_test)[:,1]
# Compute and print AUC score
print("AUC: {}".format(roc_auc_score(y_test, y_pred_prob)))
cv_auc = cross_val_score(log,X,y,cv=5,scoring='roc_auc')
# Print list of AUC scores
print("AUC scores computed using 5-fold cross-validation: {}".format(cv_auc))

## 2.  随机森林

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc=RandomForestClassifier()
rfc.fit(X_train,y_train)
y_pred=rfc.predict(X_test)

y_pred_prob = rfc.predict_proba(X_test)[:,1]

# Compute and print AUC score
print("AUC: {}".format(roc_auc_score(y_test, y_pred_prob)))

cv_auc = cross_val_score(rfc,X,y,cv=5,scoring='roc_auc')

# Print list of AUC scores
print("AUC scores computed using 5-fold cross-validation: {}".format(cv_auc))